# Week 1, Lab 1: Introduction to Search

## Welcome to AI!

In this first lab, we'll explore one of the fundamental problems in AI: **search**. Search algorithms help AI agents find solutions by exploring different possibilities.

### What You'll Learn

- What is search in AI?
- Problem representation (states, actions, goals)
- Depth-First Search (DFS)
- Breadth-First Search (BFS)
- When to use each algorithm

### Real-World Applications

- GPS navigation
- Game playing (chess, Go)
- Puzzle solving
- Robot path planning
- Web crawling

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from typing import List, Tuple, Set, Optional

# Make plots look nice
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Understanding Search Problems

A search problem consists of:

1. **Initial State**: Where we start
2. **Actions**: What we can do from each state
3. **Transition Model**: Where actions lead us
4. **Goal Test**: How to know if we've reached the goal
5. **Path Cost**: The cost of taking actions (optional)

### Example: Simple Maze

Let's represent a simple maze as a grid where:
- `0` = open space (can move)
- `1` = wall (cannot move)
- `S` = start position
- `G` = goal position

In [ ]:
# Create a simple 5x5 maze
maze = np.array([
    [0, 0, 1, 0, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 0, 0, 1],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0]
])

start = (0, 0)  # Top-left
goal = (4, 4)   # Bottom-right

def visualize_maze(maze, start, goal, path=None):
    """Visualize the maze with start, goal, and optional solution path."""
    plt.figure(figsize=(8, 8))
    
    # Create a colorful visualization
    display = np.copy(maze).astype(float)
    
    # Mark start and goal
    display[start] = 0.3  # Start in light color
    display[goal] = 0.7   # Goal in different color
    
    # Mark path if provided
    if path:
        for pos in path[1:-1]:  # Skip start and goal
            display[pos] = 0.5
    
    plt.imshow(display, cmap='RdYlGn_r', interpolation='nearest')
    plt.colorbar(label='0=Open, 1=Wall')
    
    # Add grid
    for i in range(maze.shape[0] + 1):
        plt.axhline(i - 0.5, color='black', linewidth=0.5)
    for j in range(maze.shape[1] + 1):
        plt.axvline(j - 0.5, color='black', linewidth=0.5)
    
    # Labels
    plt.text(start[1], start[0], 'S', ha='center', va='center', 
             fontsize=20, fontweight='bold', color='blue')
    plt.text(goal[1], goal[0], 'G', ha='center', va='center', 
             fontsize=20, fontweight='bold', color='green')
    
    plt.title('Maze: S=Start, G=Goal, Black=Walls')
    plt.tight_layout()
    plt.show()

# Visualize our maze
visualize_maze(maze, start, goal)

## 2. Actions and Neighbors

From any position in the maze, we can move in 4 directions: up, down, left, right.
But we can only move to valid positions (not walls, not out of bounds).

In [ ]:
def get_neighbors(maze: np.ndarray, pos: Tuple[int, int]) -> List[Tuple[int, int]]:
    """Get valid neighboring positions (up, down, left, right)."""
    rows, cols = maze.shape
    row, col = pos
    neighbors = []
    
    # Four possible moves: up, down, left, right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    for dr, dc in directions:
        new_row, new_col = row + dr, col + dc
        
        # Check if the new position is valid
        if (0 <= new_row < rows and 
            0 <= new_col < cols and 
            maze[new_row, new_col] == 0):
            neighbors.append((new_row, new_col))
    
    return neighbors

# Test: Find neighbors of start position
print(f"Start position: {start}")
print(f"Neighbors: {get_neighbors(maze, start)}")

# Test: Find neighbors of a middle position
test_pos = (2, 2)
print(f"\nPosition {test_pos}")
print(f"Neighbors: {get_neighbors(maze, test_pos)}")

## 3. Depth-First Search (DFS)

DFS explores as far as possible along each branch before backtracking.

### How it works:
1. Start at the initial state
2. Explore one neighbor completely before trying others
3. Use a **stack** (LIFO - Last In First Out)
4. Backtrack when we hit a dead end

### Pros:
- Memory efficient (stores only current path)
- Can find a solution quickly if lucky

### Cons:
- Might not find the shortest path
- Can get stuck in infinite loops (we'll use visited set to prevent this)

In [ ]:
def dfs(maze: np.ndarray, start: Tuple[int, int], goal: Tuple[int, int]) -> Optional[List[Tuple[int, int]]]:
    """
    Depth-First Search to find a path from start to goal.
    
    Returns:
        List of positions from start to goal, or None if no path exists
    """
    # Stack stores tuples of (current_position, path_to_current)
    stack = [(start, [start])]
    visited = set([start])
    
    nodes_explored = 0
    
    while stack:
        current, path = stack.pop()  # LIFO - Last In First Out
        nodes_explored += 1
        
        # Check if we reached the goal
        if current == goal:
            print(f"✓ DFS found a path! Explored {nodes_explored} nodes.")
            return path
        
        # Explore neighbors
        for neighbor in get_neighbors(maze, current):
            if neighbor not in visited:
                visited.add(neighbor)
                new_path = path + [neighbor]
                stack.append((neighbor, new_path))
    
    print(f"✗ DFS found no path. Explored {nodes_explored} nodes.")
    return None

# Run DFS
dfs_path = dfs(maze, start, goal)

if dfs_path:
    print(f"Path length: {len(dfs_path)} steps")
    print(f"Path: {dfs_path}")
    visualize_maze(maze, start, goal, dfs_path)

## 4. Breadth-First Search (BFS)

BFS explores all neighbors at the current depth before moving to the next depth level.

### How it works:
1. Start at the initial state
2. Explore all neighbors at distance 1, then distance 2, etc.
3. Use a **queue** (FIFO - First In First Out)
4. Guaranteed to find the shortest path!

### Pros:
- **Always finds the shortest path** (in terms of number of steps)
- Complete (will find a solution if one exists)

### Cons:
- Uses more memory (stores all nodes at current level)
- Can be slow for very deep searches

In [ ]:
def bfs(maze: np.ndarray, start: Tuple[int, int], goal: Tuple[int, int]) -> Optional[List[Tuple[int, int]]]:
    """
    Breadth-First Search to find the SHORTEST path from start to goal.
    
    Returns:
        Shortest list of positions from start to goal, or None if no path exists
    """
    # Queue stores tuples of (current_position, path_to_current)
    queue = deque([(start, [start])])
    visited = set([start])
    
    nodes_explored = 0
    
    while queue:
        current, path = queue.popleft()  # FIFO - First In First Out
        nodes_explored += 1
        
        # Check if we reached the goal
        if current == goal:
            print(f"✓ BFS found the shortest path! Explored {nodes_explored} nodes.")
            return path
        
        # Explore neighbors
        for neighbor in get_neighbors(maze, current):
            if neighbor not in visited:
                visited.add(neighbor)
                new_path = path + [neighbor]
                queue.append((neighbor, new_path))
    
    print(f"✗ BFS found no path. Explored {nodes_explored} nodes.")
    return None

# Run BFS
bfs_path = bfs(maze, start, goal)

if bfs_path:
    print(f"Path length: {len(bfs_path)} steps")
    print(f"Path: {bfs_path}")
    visualize_maze(maze, start, goal, bfs_path)

## 5. Comparing DFS and BFS

Let's compare the two algorithms side by side:

In [ ]:
# Create a comparison
print("="*60)
print("COMPARISON: DFS vs BFS")
print("="*60)

if dfs_path and bfs_path:
    print(f"DFS path length: {len(dfs_path)} steps")
    print(f"BFS path length: {len(bfs_path)} steps")
    print()
    
    if len(bfs_path) < len(dfs_path):
        print("✓ BFS found a shorter path!")
        print(f"  BFS is {len(dfs_path) - len(bfs_path)} steps shorter.")
    elif len(bfs_path) == len(dfs_path):
        print("✓ Both found paths of equal length!")
    
print()
print("Key Differences:")
print("  DFS: Uses stack (LIFO) - goes deep first")
print("  BFS: Uses queue (FIFO) - goes wide first")
print()
print("  DFS: Memory efficient but may find longer paths")
print("  BFS: Guarantees shortest path but uses more memory")

## 6. Exercise: Try It Yourself!

Now it's your turn to experiment!

In [ ]:
# Exercise 1: Create your own maze
# Hint: 0 = open, 1 = wall
my_maze = np.array([
    [0, 0, 0, 0, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
])

my_start = (0, 0)
my_goal = (4, 4)

# Visualize your maze
visualize_maze(my_maze, my_start, my_goal)

# Test both algorithms
print("\nRunning DFS:")
my_dfs_path = dfs(my_maze, my_start, my_goal)

print("\nRunning BFS:")
my_bfs_path = bfs(my_maze, my_start, my_goal)

# Visualize solutions
if my_dfs_path:
    visualize_maze(my_maze, my_start, my_goal, my_dfs_path)
if my_bfs_path:
    visualize_maze(my_maze, my_start, my_goal, my_bfs_path)

## 7. Key Takeaways

1. **Search is fundamental to AI** - many problems can be framed as search problems

2. **DFS (Depth-First Search)**:
   - Uses a stack (LIFO)
   - Memory efficient
   - May not find shortest path
   - Good for: deep trees, memory constraints

3. **BFS (Breadth-First Search)**:
   - Uses a queue (FIFO)
   - Guarantees shortest path
   - Uses more memory
   - Good for: finding shortest path, shallow solutions

4. **Problem Representation Matters**:
   - States, actions, goal test are crucial
   - Good representation makes algorithms more efficient

## Next Up

In Lab 2, we'll explore:
- Uniform Cost Search (considering action costs)
- Greedy Search (using heuristics)
- A* Search (the best of both worlds!)

## Additional Challenges

1. Modify the code to count how many nodes each algorithm explores
2. Create a larger maze (10x10 or 20x20) and compare performance
3. Allow diagonal movement (8 directions instead of 4)
4. Add a cost to each move and track total path cost
5. Visualize the search process step-by-step (animation)

Happy coding!